# GPT2 Augmentation technique

### Libraries

In [1]:
# Connect drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd

import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Load model

In [ ]:
# Load the GPT-2 model and tokenizer
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

### Load set

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/NCU/Data Science and Machine Learning/Assignment 3/Datasets/train.csv')
rwtrain = train.drop(train.columns[0], axis=1)

In [ ]:
rwtrain.head()

,TEXT,LABEL
0,director dirk shafer and co-writer greg hinton...,0
1,"a charming , quirky and leisurely paced scotti...",1
2,"the price was good , and came quickly though ...",1
3,i was looking forward to this game for a coupl...,0
4,arguably the year 's silliest and most incoher...,0


### Preprocessing

In [ ]:
def preprocess_text(text):
    # convert text to lowercase
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # tokenize the text into words
    words = nltk.word_tokenize(text)
    # remove stop words
    words = [word for word in words if word not in stopwords.words('english')]
    # lemmatize the remaining words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    # join the words back into a single string
    text = " ".join(words)
    return text

In [ ]:
rwtrain['TEXT'] = rwtrain['TEXT'].apply(preprocess_text)

In [ ]:
rwtrain.head()

,TEXT,LABEL
0,director dirk shafer cowriter greg hinton ride...,0
1,charming quirky leisurely paced scottish comed...,1
2,price good came quickly though prime membership,1
3,looking forward game couple numextend,0
4,arguably year silliest incoherent movie,0


### Autmentation strategy

In [ ]:
def generate_synthetic_samples(prompt, num_samples, temperature = 1.1):
    inputs = tokenizer.encode(prompt, return_tensors='pt', truncation=True, max_length=43)
    synthetic_samples = []
    print(f'Generating {num_samples} samples...')

    for _ in range(num_samples):
        outputs = model.generate(inputs, max_length=43, pad_token_id=tokenizer.eos_token_id, do_sample=True, temperature = temperature)
        synthetic_sample = tokenizer.decode(outputs[0], skip_special_tokens=True)
        synthetic_samples.append(synthetic_sample)

    return synthetic_samples

### Generating data

In [ ]:
# 97' with 2 augmentations with CPU
# 207' with 4 augmentations with CPU
# Generate synthetic samples
augmentation_factor = 2  # Set the desired number of synthetic samples per original sample
augmented_data = []
for index, row in rwtrain.iterrows():
    original_sample = row['TEXT']  # Assuming 'TEXT' is the column name for sentences
    original_label = row['LABEL']  # Assuming 'LABEL' is the column name for labels

    print(f'Processing row index: {index}...')

    synthetic_samples = generate_synthetic_samples(original_sample, augmentation_factor)
    augmented_data.append({'TEXT': original_sample, 'LABEL': original_label})
    
    for synthetic_sample in synthetic_samples:
        augmented_data.append({'TEXT': synthetic_sample, 'LABEL': original_label})


### Constructing augmented dataset

In [ ]:
# Combine original and synthetic data
augmented_dataset = pd.DataFrame(augmented_data)

In [ ]:
# Save the augmented dataset as a CSV file
augmented_dataset.to_csv('2T11_GPT_train.csv', index=False)